# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20901993


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:07<01:49,  3.65s/it]

Rendering models:   9%|▉         | 3/32 [00:07<01:18,  2.72s/it]

Rendering models:  12%|█▎        | 4/32 [00:08<00:57,  2.06s/it]

Rendering models:  16%|█▌        | 5/32 [00:08<00:41,  1.55s/it]

...

Rendering models:  91%|█████████ | 29/32 [00:14<00:00,  4.25it/s]

Rendering models:  94%|█████████▍| 30/32 [00:14<00:00,  4.29it/s]

Rendering models:  97%|█████████▋| 31/32 [00:15<00:00,  3.59it/s]

Rendering models: 100%|██████████| 32/32 [00:15<00:00,  4.43it/s]

equidad1                              0.051416
jstephenson87                         0.001378
not-logged-in-6f58cd197f9447fe73f1    0.000594
Mark_Moffatt                          0.001469
ElisabethB                            0.001333
Jes8x                                 0.000761
cmk24                                 0.000452
rjsmethurst                           0.000612
not-logged-in-7c0e3b1ae8dc7fbb6b66    0.010756
Evan.wickham                          0.024294
crush202020                           0.000541
acapirala                             0.000621
Dpwillet                              0.000961
not-logged-in-4c6d97421fdb5b2fcda3    0.013265
Lavadude                              0.022079
AllyExcell                            0.033996
Bambura                               0.001048
jzeiszler                             0.001109
Rbaker48                              0.002037
simon_harding                         0.001444
swalla46                              0.000656
mcooke27     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())